# 病床姿勢監測 - LabelMe 標註轉換

**專案**: 熱成像超解析度 + 姿勢辨識  
**作者**: rochi190  
**日期**: 2025-11-03  

## 功能
1. 讀取 LabelMe JSON 標註
2. 解析人體區域 polygon
3. 生成 binary mask（人體區域）
4. 建立訓練用 CSV（pair_id, pose_label, mask_path）
5. 統計標註分佈
6. 視覺化標註結果

## 輸入
- `output/labelme_project/annotations/` - LabelMe JSON 檔案
- `output/labelme_project/image_metadata.csv` - 圖片 metadata

## 輸出
- `output/training_dataset/person_masks/` - 人體 mask (PNG)
- `output/training_dataset/pose_labels.csv` - 姿勢標籤
- `output/training_dataset/label_statistics.png` - 統計圖表

---
## Step 0: 環境設定與套件匯入

In [5]:
import os
import sys
import json
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from typing import List, Dict, Tuple, Optional, Any
from tqdm import tqdm
from collections import Counter
import warnings
warnings.filterwarnings('ignore')
from collections import Counter, defaultdict
# Matplotlib 設定
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'SimHei', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams['figure.dpi'] = 100

# Seaborn 樣式
sns.set_style('whitegrid')
sns.set_palette('husl')

print("="*70)
print("病床姿勢監測系統 - LabelMe 標註轉換模組")
print("="*70)
print(f"執行時間: {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"使用者: rochi190")
print(f"Python 版本: {sys.version}")
print(f"NumPy 版本: {np.__version__}")
print(f"OpenCV 版本: {cv2.__version__}")
print("="*70)

病床姿勢監測系統 - LabelMe 標註轉換模組
執行時間: 2025-11-04 18:15:23
使用者: rochi190
Python 版本: 3.11.13 (main, Jun  5 2025, 13:12:00) [GCC 11.2.0]
NumPy 版本: 1.26.4
OpenCV 版本: 4.10.0


---
## Step 1: 路徑配置

In [ ]:
# ========================================
# 路徑配置
# ========================================
BASE_DIR = Path(os.getcwd())
OUTPUT_DIR = BASE_DIR / 'output'

# LabelMe 專案目錄
LABELME_DIR = OUTPUT_DIR / 'labelme_project'
IMAGES_DIR = LABELME_DIR / 'images'
ANNOTATIONS_DIR = LABELME_DIR / 'annotations'
METADATA_CSV = LABELME_DIR / 'image_metadata.csv'

# 輸出目錄
TRAINING_DIR = OUTPUT_DIR / 'training_dataset'
MASKS_DIR = TRAINING_DIR / 'person_masks'

# 🔧 改為 adaptive_pairing
ADAPTIVE_DIR = OUTPUT_DIR / 'adaptive_pairing'  # ← 這裡
ALIGNED_DIR = OUTPUT_DIR / 'aligned_dataset'    # ← 保留（用於 03 訓練時）

# 建立輸出目錄
for directory in [TRAINING_DIR, MASKS_DIR]:
    directory.mkdir(parents=True, exist_ok=True)

# ========================================
# 類別映射（只保留 3 個有效姿勢）
# ========================================
CLASS_MAPPING = {
    'lying': 0,      # 躺著
    'sitting': 1,    # 坐著
    'fallen': 2,     # 跌倒
    # 'empty': 移除（視野問題）
    # 'uncertain': 移除（品質不佳）
}

CLASS_NAMES = list(CLASS_MAPPING.keys())
VALID_POSES = ['lying', 'sitting', 'fallen']

print(f"\n🏷️ 訓練用姿勢類別:")
for name, idx in CLASS_MAPPING.items():
    print(f"  {idx}. {name}")

print(f"\n⚠️ 過濾的類別: empty, uncertain")

# ========================================
# 顯示配置
# ========================================
print(f"\n📁 路徑檢查:")
print(f"  LabelMe 目錄: {'✓' if LABELME_DIR.exists() else '✗'} {LABELME_DIR}")
print(f"  圖片目錄: {'✓' if IMAGES_DIR.exists() else '✗'} {IMAGES_DIR}")
print(f"  標註目錄: {'✓' if ANNOTATIONS_DIR.exists() else '✗'} {ANNOTATIONS_DIR}")
print(f"  Metadata: {'✓' if METADATA_CSV.exists() else '✗'} {METADATA_CSV}")
print(f"  Adaptive Pairing: {'✓' if ADAPTIVE_DIR.exists() else '✗'} {ADAPTIVE_DIR}")

print(f"\n🏷️ 姿勢類別:")
for name, idx in CLASS_MAPPING.items():
    print(f"  {idx}. {name}")

# 檢查標註檔案
if ANNOTATIONS_DIR.exists():
    json_files = list(ANNOTATIONS_DIR.glob('*.json'))
    print(f"\n📊 標註檔案: {len(json_files)} 個")
else:
    print(f"\n⚠️ 標註目錄不存在，請先執行 LabelMe 標註")


🏷️ 訓練用姿勢類別:
  0. lying
  1. sitting
  2. fallen

⚠️ 過濾的類別: empty, uncertain

📁 路徑檢查:
  LabelMe 目錄: ✓ /home/gary/claude4.5/output/labelme_project
  圖片目錄: ✓ /home/gary/claude4.5/output/labelme_project/images
  標註目錄: ✓ /home/gary/claude4.5/output/labelme_project/annotations
  Metadata: ✗ /home/gary/claude4.5/output/labelme_project/image_metadata.csv

🏷️ 姿勢類別:
  0. lying
  1. sitting
  2. fallen

📊 標註檔案: 0 個


---
## Step 2: 解析 LabelMe JSON

In [ ]:
def parse_labelme_json(json_path: Path) -> Dict[str, Any]:
    """
    解析 LabelMe JSON 檔案
    
    Args:
        json_path: JSON 檔案路徑
        
    Returns:
        包含標註資訊的字典
    """
    with open(json_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    labels = []
    polygons = []
    
    for shape in data.get('shapes', []):
        label = shape['label']
        points = np.array(shape['points'], dtype=np.int32)
        
        labels.append(label)
        polygons.append(points)
    
    return {
        'image_path': data['imagePath'],
        'labels': labels,
        'polygons': polygons,
        'image_height': data['imageHeight'],
        'image_width': data['imageWidth'],
        'version': data.get('version', 'unknown')
    }

def create_mask_from_polygon(points: np.ndarray, height: int, width: int) -> np.ndarray:
    """
    從 polygon 點建立 binary mask
    
    Args:
        points: Polygon 頂點座標 (N, 2)
        height: 圖片高度
        width: 圖片寬度
        
    Returns:
        Binary mask (uint8, 0 或 255)
    """
    mask = np.zeros((height, width), dtype=np.uint8)
    cv2.fillPoly(mask, [points], 255)
    return mask

# 測試解析功能
json_files = sorted(ANNOTATIONS_DIR.glob('*.json'))

if json_files:
    print(f"\n測試解析第一個標註檔案...")
    test_annotation = parse_labelme_json(json_files[0])
    
    print(f"\n✅ 解析成功")
    print(f"  圖片: {test_annotation['image_path']}")
    print(f"  解析度: {test_annotation['image_width']} × {test_annotation['image_height']}")
    print(f"  標註數量: {len(test_annotation['labels'])}")
    print(f"  標籤: {test_annotation['labels']}")
    print(f"  LabelMe 版本: {test_annotation['version']}")
else:
    print(f"\n❌ 未找到標註檔案")

---
## Step 3: 批次轉換標註

In [ ]:
def convert_all_annotations(
    annotations_dir: Path,
    metadata_csv: Path,
    output_masks_dir: Path
) -> pd.DataFrame:
    """
    轉換所有 LabelMe 標註為訓練格式
    
    修改：過濾 empty 和 uncertain 標註
    """
    json_files = sorted(annotations_dir.glob('*.json'))
    
    if not json_files:
        raise FileNotFoundError(f"❌ 未找到標註檔案於 {annotations_dir}")
    
    print(f"\n{'='*70}")
    print(f"轉換 LabelMe 標註")
    print(f"{'='*70}")
    print(f"找到 {len(json_files)} 個標註檔案")
    
    # 載入 metadata
    if metadata_csv.exists():
        metadata_df = pd.read_csv(metadata_csv)
        print(f"載入 metadata: {len(metadata_df)} 筆")
    else:
        print(f"⚠️ 未找到 metadata")
        metadata_df = None
    
    results = []
    skipped = []
    skipped_by_pose = defaultdict(int)  # 統計各類別被跳過的數量
    
    print(f"\n開始轉換（只保留 {VALID_POSES}）...")
    for json_path in tqdm(json_files, desc="轉換標註", unit="file"):
        try:
            # 解析 JSON
            annotation = parse_labelme_json(json_path)
            
            # 找對應的 metadata
            image_name = Path(annotation['image_path']).name
            
            if metadata_df is not None:
                meta_row = metadata_df[metadata_df['filename'] == image_name]
                
                if meta_row.empty:
                    skipped.append({'file': json_path.name, 'reason': 'metadata not found'})
                    continue
                
                pair_id = meta_row.iloc[0]['pair_id']
                timestamp = meta_row.iloc[0]['timestamp']
                rgb_frame_idx = meta_row.iloc[0]['rgb_frame_idx']
            else:
                # 從檔名推斷 pair_id
                pair_id = image_name.replace('frame_', '').replace('.jpg', '')
                pair_id = f"pair_{pair_id}"
                timestamp = None
                rgb_frame_idx = None
            
            # 處理每個標註（通常只有一個人）
            for idx, (label, polygon) in enumerate(zip(annotation['labels'], annotation['polygons'])):
                
                # 🔧 過濾無效姿勢
                if label not in VALID_POSES:
                    skipped_by_pose[label] += 1
                    skipped.append({
                        'file': json_path.name, 
                        'reason': f'filtered pose: {label}'
                    })
                    continue
                
                # 建立 mask
                mask = create_mask_from_polygon(
                    polygon,
                    annotation['image_height'],
                    annotation['image_width']
                )
                
                # 儲存 mask
                mask_filename = f"{pair_id}_mask.png"
                mask_path = output_masks_dir / mask_filename
                cv2.imwrite(str(mask_path), mask)
                
                # 記錄結果
                results.append({
                    'pair_id': pair_id,
                    'timestamp': timestamp,
                    'rgb_frame_idx': rgb_frame_idx,
                    'image_file': image_name,
                    'mask_file': mask_filename,
                    'mask_path': str(mask_path.relative_to(output_masks_dir.parent)),
                    'pose_label': label,
                    'pose_class': CLASS_MAPPING.get(label, -1),
                    'polygon_points': len(polygon),
                    'mask_area': np.sum(mask > 0),
                    'image_width': annotation['image_width'],
                    'image_height': annotation['image_height']
                })
        
        except Exception as e:
            skipped.append({'file': json_path.name, 'reason': str(e)})
    
    # 建立 DataFrame
    results_df = pd.DataFrame(results)
    
    print(f"\n✅ 轉換完成")
    print(f"  成功: {len(results)} 個標註")
    print(f"  跳過: {len(skipped)} 個檔案")
    
    # 🔧 顯示過濾統計
    if skipped_by_pose:
        print(f"\n⚠️ 依姿勢過濾統計:")
        for pose, count in sorted(skipped_by_pose.items()):
            print(f"    - {pose}: {count} 個")
    
    if skipped:
        print(f"\n  其他跳過原因（前 5 個）:")
        for item in skipped[:5]:
            if not item['reason'].startswith('filtered pose'):
                print(f"    - {item['file']}: {item['reason']}")
        if len(skipped) > 5:
            other_skipped = [s for s in skipped if not s['reason'].startswith('filtered pose')]
            if len(other_skipped) > 5:
                print(f"    ... 還有 {len(other_skipped)-5} 個")
    
    print(f"\n📊 有效標註分佈:")
    if len(results_df) > 0:
        print(results_df['pose_label'].value_counts())
    else:
        print("  ⚠️ 無有效標註")
    
    print("="*70)
    
    return results_df

# ========================================
# 執行轉換
# ========================================
labels_df = convert_all_annotations(ANNOTATIONS_DIR, METADATA_CSV, MASKS_DIR)

# 儲存結果
output_csv = TRAINING_DIR / 'pose_labels.csv'
labels_df.to_csv(output_csv, index=False, encoding='utf-8-sig')

print(f"\n💾 標籤已儲存: {output_csv}")
print(f"\n前 5 筆資料:")
print(labels_df.head())

---
## Step 4: 統計分析

In [ ]:
def analyze_label_distribution(labels_df: pd.DataFrame) -> None:
    """
    分析標註分佈
    """
    print(f"\n{'='*70}")
    print(f"標註統計分析")
    print(f"{'='*70}")
    
    # 基本統計
    print(f"\n📊 基本統計:")
    print(f"  總標註數: {len(labels_df):,}")
    print(f"  唯一 pair 數: {labels_df['pair_id'].nunique():,}")
    
    # 姿勢分佈
    print(f"\n🏷️ 姿勢分佈:")
    pose_counts = labels_df['pose_label'].value_counts()
    for pose, count in pose_counts.items():
        percentage = count / len(labels_df) * 100
        print(f"  {pose:12s}: {count:5d} ({percentage:5.1f}%)")
    
    # Mask 區域統計
    if 'mask_area' in labels_df.columns:
        print(f"\n📐 Mask 區域統計 (pixels):")
        total_pixels = labels_df['image_width'] * labels_df['image_height']
        mask_ratio = labels_df['mask_area'] / total_pixels * 100
        
        print(f"  平均面積: {labels_df['mask_area'].mean():,.0f} pixels")
        print(f"  中位數: {labels_df['mask_area'].median():,.0f} pixels")
        print(f"  最小: {labels_df['mask_area'].min():,.0f} pixels")
        print(f"  最大: {labels_df['mask_area'].max():,.0f} pixels")
        print(f"  平均佔比: {mask_ratio.mean():.1f}% of image")
    
    print("="*70)

# 執行分析
analyze_label_distribution(labels_df)

---
## Step 5: 視覺化

In [ ]:
def visualize_label_statistics(labels_df: pd.DataFrame, save_dir: Path) -> None:
    """
    視覺化標註統計
    """
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    
    # 1. 姿勢分佈（柱狀圖）
    pose_counts = labels_df['pose_label'].value_counts()
    axes[0, 0].bar(range(len(pose_counts)), pose_counts.values, color='steelblue', edgecolor='black')
    axes[0, 0].set_xticks(range(len(pose_counts)))
    axes[0, 0].set_xticklabels(pose_counts.index, rotation=45, ha='right')
    axes[0, 0].set_ylabel('Count')
    axes[0, 0].set_title('Pose Label Distribution (Bar Chart)')
    axes[0, 0].grid(axis='y', alpha=0.3)
    
    # 加上數值標籤
    for i, v in enumerate(pose_counts.values):
        axes[0, 0].text(i, v + 0.5, str(v), ha='center', fontweight='bold')
    
    # 2. 姿勢分佈（圓餅圖）
    axes[0, 1].pie(pose_counts.values, labels=pose_counts.index, autopct='%1.1f%%', startangle=90)
    axes[0, 1].set_title('Pose Label Distribution (Pie Chart)')
    
    # 3. Mask 面積分佈
    if 'mask_area' in labels_df.columns:
        axes[1, 0].hist(labels_df['mask_area'], bins=30, color='coral', edgecolor='black', alpha=0.7)
        axes[1, 0].axvline(labels_df['mask_area'].mean(), color='red', linestyle='--', 
                          label=f"Mean: {labels_df['mask_area'].mean():.0f}")
        axes[1, 0].axvline(labels_df['mask_area'].median(), color='green', linestyle='--', 
                          label=f"Median: {labels_df['mask_area'].median():.0f}")
        axes[1, 0].set_xlabel('Mask Area (pixels)')
        axes[1, 0].set_ylabel('Frequency')
        axes[1, 0].set_title('Mask Area Distribution')
        axes[1, 0].legend()
        axes[1, 0].grid(alpha=0.3)
    
    # 4. 各姿勢的 Mask 面積箱型圖
    if 'mask_area' in labels_df.columns:
        pose_labels = labels_df['pose_label'].unique()
        data_for_box = [labels_df[labels_df['pose_label'] == pose]['mask_area'].values 
                       for pose in pose_labels]
        
        bp = axes[1, 1].boxplot(data_for_box, labels=pose_labels, patch_artist=True)
        for patch in bp['boxes']:
            patch.set_facecolor('lightblue')
        axes[1, 1].set_ylabel('Mask Area (pixels)')
        axes[1, 1].set_title('Mask Area by Pose Label')
        axes[1, 1].tick_params(axis='x', rotation=45)
        axes[1, 1].grid(axis='y', alpha=0.3)
    
    plt.tight_layout()
    
    save_path = save_dir / 'label_statistics.png'
    plt.savefig(save_path, dpi=150, bbox_inches='tight')
    print(f"\n✅ 統計圖表已儲存: {save_path}")
    plt.show()

def visualize_sample_masks(
    labels_df: pd.DataFrame,
    images_dir: Path,
    masks_dir: Path,
    save_dir: Path,
    num_samples: int = 8
) -> None:
    """
    視覺化樣本 mask
    """
    # 每個類別取樣
    pose_labels = labels_df['pose_label'].unique()
    samples_per_class = max(1, num_samples // len(pose_labels))
    
    sample_rows = []
    for pose in pose_labels:
        pose_samples = labels_df[labels_df['pose_label'] == pose].sample(
            min(samples_per_class, len(labels_df[labels_df['pose_label'] == pose]))
        )
        sample_rows.append(pose_samples)
    
    samples_df = pd.concat(sample_rows)
    
    n_samples = len(samples_df)
    n_cols = 4
    n_rows = (n_samples + n_cols - 1) // n_cols
    
    fig, axes = plt.subplots(n_rows, n_cols * 2, figsize=(16, n_rows * 3))
    if n_rows == 1:
        axes = axes.reshape(1, -1)
    
    for idx, (_, row) in enumerate(samples_df.iterrows()):
        row_idx = idx // n_cols
        col_idx = (idx % n_cols) * 2
        
        # 讀取 RGB 圖片
        rgb_path = images_dir / row['image_file']
        if rgb_path.exists():
            rgb = cv2.imread(str(rgb_path))
            rgb = cv2.cvtColor(rgb, cv2.COLOR_BGR2RGB)
        else:
            rgb = np.zeros((480, 640, 3), dtype=np.uint8)
        
        # 讀取 mask
        mask_path = masks_dir / row['mask_file']
        if mask_path.exists():
            mask = cv2.imread(str(mask_path), cv2.IMREAD_GRAYSCALE)
        else:
            mask = np.zeros((480, 640), dtype=np.uint8)
        
        # 顯示 RGB
        axes[row_idx, col_idx].imshow(rgb)
        axes[row_idx, col_idx].set_title(f"{row['pose_label']}\n{row['pair_id']}")
        axes[row_idx, col_idx].axis('off')
        
        # 顯示 Mask overlay
        overlay = rgb.copy()
        overlay[mask > 0] = [255, 0, 0]  # 紅色
        blended = cv2.addWeighted(rgb, 0.7, overlay, 0.3, 0)
        
        axes[row_idx, col_idx + 1].imshow(blended)
        axes[row_idx, col_idx + 1].set_title(f"Mask Overlay\nArea: {row['mask_area']:,.0f}px")
        axes[row_idx, col_idx + 1].axis('off')
    
    # 隱藏多餘的 subplot
    for idx in range(n_samples, n_rows * n_cols):
        row_idx = idx // n_cols
        col_idx = (idx % n_cols) * 2
        axes[row_idx, col_idx].axis('off')
        axes[row_idx, col_idx + 1].axis('off')
    
    plt.suptitle('Sample Annotations with Mask Overlay', fontsize=14, fontweight='bold')
    plt.tight_layout()
    
    save_path = save_dir / 'sample_masks.png'
    plt.savefig(save_path, dpi=120, bbox_inches='tight')
    print(f"\n✅ 樣本 Mask 已儲存: {save_path}")
    plt.show()

# ========================================
# 執行視覺化
# ========================================
visualize_label_statistics(labels_df, TRAINING_DIR)
visualize_sample_masks(labels_df, IMAGES_DIR, MASKS_DIR, TRAINING_DIR)

---
## Step 6: 合併對齊資料

In [ ]:
def merge_with_aligned_data(
    labels_df: pd.DataFrame,
    aligned_dir: Path
) -> pd.DataFrame:
    """
    合併標籤與對齊後的 Thermal 資料
    
    Args:
        labels_df: 標籤 DataFrame
        aligned_dir: 對齊資料目錄
        
    Returns:
        合併後的 DataFrame
    """
    aligned_metadata = aligned_dir / 'metadata.csv'
    
    if not aligned_metadata.exists():
        print(f"⚠️ 未找到對齊資料: {aligned_metadata}")
        return labels_df
    
    print(f"\n{'='*70}")
    print(f"合併對齊資料")
    print(f"{'='*70}")
    
    aligned_df = pd.read_csv(aligned_metadata)
    print(f"載入對齊資料: {len(aligned_df)} 筆")
    
    # 合併
    merged_df = labels_df.merge(
        aligned_df[['pair_id', 'thermal_file', 'thermal_error_ms', 'rgb_error_ms']],
        on='pair_id',
        how='left'
    )
    
    print(f"\n✅ 合併完成")
    print(f"  標籤數: {len(labels_df)}")
    print(f"  對齊數: {len(aligned_df)}")
    print(f"  合併數: {len(merged_df)}")
    print(f"  成功匹配: {merged_df['thermal_file'].notna().sum()}")
    
    print("="*70)
    
    return merged_df

# 執行合併
if ALIGNED_DIR.exists():
    merged_df = merge_with_aligned_data(labels_df, ALIGNED_DIR)
    
    # 儲存合併結果
    merged_csv = TRAINING_DIR / 'pose_labels_with_thermal.csv'
    merged_df.to_csv(merged_csv, index=False, encoding='utf-8-sig')
    print(f"\n💾 合併資料已儲存: {merged_csv}")
    
    print(f"\n前 5 筆資料:")
    print(merged_df.head())
else:
    print(f"\n⚠️ 對齊資料目錄不存在: {ALIGNED_DIR}")
    merged_df = labels_df

---
## Step 7: 產生訓練摘要

In [ ]:
# 產生訓練摘要
summary = {
    'total_samples': len(merged_df),
    'unique_pairs': merged_df['pair_id'].nunique(),
    'pose_distribution': merged_df['pose_label'].value_counts().to_dict(),
    'class_counts': {name: (merged_df['pose_class'] == idx).sum() 
                    for name, idx in CLASS_MAPPING.items()},
    'paths': {
        'labels_csv': str(output_csv),
        'merged_csv': str(TRAINING_DIR / 'pose_labels_with_thermal.csv') if 'merged_df' in locals() else None,
        'masks_dir': str(MASKS_DIR),
        'aligned_dir': str(ALIGNED_DIR)
    },
    'statistics': {
        'mean_mask_area': float(merged_df['mask_area'].mean()) if 'mask_area' in merged_df.columns else None,
        'median_mask_area': float(merged_df['mask_area'].median()) if 'mask_area' in merged_df.columns else None
    }
}

summary_json = TRAINING_DIR / 'training_summary.json'
with open(summary_json, 'w', encoding='utf-8') as f:
    json.dump(summary, f, indent=2, ensure_ascii=False)

print(f"\n{'='*70}")
print(f"訓練資料摘要")
print(f"{'='*70}")
print(json.dumps(summary, indent=2, ensure_ascii=False))
print(f"\n💾 摘要已儲存: {summary_json}")
print("="*70)

print(f"\n✅ 所有轉換完成！")
print(f"\n下一步: 執行 03_train_semantic_guided_sr.ipynb 開始訓練")